In [2]:
import mysql.connector
import requests
import time
import os
import mimetypes

db_params = {
    'host': '192.168.100.200',
    'user': 'ecorradini',
    'password': 'birillo',
    'database': 'threads'
}

def connect_to_db():
    return mysql.connector.connect(**db_params)

def fetch_new_images_post_old(cursor):
    query = """
        SELECT p.post_url, p.image_url
        FROM post p
        LEFT JOIN images i ON p.post_url = i.post_url
        WHERE p.image_url != '' AND i.image_path IS NULL
    """
    cursor.execute(query)
    return cursor.fetchall()

def fetch_new_images_comment(cursor):
    query = """
        SELECT c.url, c.image
        FROM comment c
        LEFT JOIN images i ON c.url = i.post_url
        WHERE c.image != '' AND i.image_path IS NULL
    """
    cursor.execute(query)
    return cursor.fetchall()

def fetch_new_images_post(cursor):
    query = """
        SELECT d.url, d.image_video_url
        FROM data d
        LEFT JOIN images i ON d.url = i.post_url
        WHERE d.image_video_url != '' AND i.image_path IS NULL
    """
    cursor.execute(query)
    return cursor.fetchall()

def download_image(db_connection, cursor, url, post_url, download_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        content_type = response.headers['Content-Type']
        extension = mimetypes.guess_extension(content_type) or '.jpg'
        filename = post_url.replace('https://www.threads.net/', '').replace('/post/', '_') + extension
        filepath = os.path.join(download_path, filename)

        with open(filepath, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        # Insert the downloaded image path into the images table
        insert_query = "INSERT INTO images (post_url, image_path) VALUES (%s, %s)"
        cursor.execute(insert_query, (post_url, filepath))
        db_connection.commit()

In [ ]:
while True:
    db_connection = connect_to_db()
    cursor = db_connection.cursor()
    new_images = fetch_new_images_post(cursor)
    new_images.extend(fetch_new_images_post_old(cursor))
    new_images.extend(fetch_new_images_comment(cursor))
    print(f"TO DOWNLOAD: {len(new_images)}")
    for post_url, image_url in new_images:
        try:
            download_image(db_connection,cursor,image_url,post_url,'/mnt/images_to_upload/')
        except Exception as e:
            continue
    cursor.close()
    db_connection.close()

TO DOWNLOAD: 3984
